# Tree Mapping with SAMGeo and Segment Anything Model 2 (SAM 2)

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/tree_mapping.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/tree_mapping.ipynb)

This notebook shows how to segment trees from aerial imagery with the Segment Anything Model 2 (SAM 2).

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [1]:
# %pip install segment-geospatial

In [2]:
import leafmap
from samgeo import SamGeo2

## Create an interactive map

In [25]:
m = leafmap.Map(center=[29.718188, -95.399490], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[29.718188, -95.39949], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [26]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-77.300, 38.834, -77.298, 38.833]

In [27]:
image = "Image.tif"
leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True
)

Downloaded image 1/651
Downloaded image 2/651
Downloaded image 3/651
Downloaded image 4/651
Downloaded image 5/651
Downloaded image 6/651
Downloaded image 7/651
Downloaded image 8/651
Downloaded image 9/651
Downloaded image 10/651
Downloaded image 11/651
Downloaded image 12/651
Downloaded image 13/651
Downloaded image 14/651
Downloaded image 15/651
Downloaded image 16/651
Downloaded image 17/651
Downloaded image 18/651
Downloaded image 19/651
Downloaded image 20/651
Downloaded image 21/651
Downloaded image 22/651
Downloaded image 23/651
Downloaded image 24/651
Downloaded image 25/651
Downloaded image 26/651
Downloaded image 27/651
Downloaded image 28/651
Downloaded image 29/651
Downloaded image 30/651
Downloaded image 31/651
Downloaded image 32/651
Downloaded image 33/651
Downloaded image 34/651
Downloaded image 35/651
Downloaded image 36/651
Downloaded image 37/651
Downloaded image 38/651
Downloaded image 39/651
Downloaded image 40/651
Downloaded image 41/651
Downloaded image 42/651
D

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [28]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=6937457.0, center=[29.717149999999997, -95.40289999999999], controls=(ZoomControl(options=['positio…

## Initialize SAM class

Set `automatic=False` to enable the `SAM2ImagePredictor`.

In [10]:
sam = SamGeo2(
    model_id="sam2-hiera-large",
    automatic=False,
)

sam2_hiera_large.pt:   0%|          | 0.00/898M [00:00<?, ?B/s]

Specify the image to segment.

In [29]:
sam.set_image(image)

Display the map. Use the drawing tools to draw some rectangles around the features you want to extract, such as trees, buildings.

In [30]:
m

Map(bottom=3468932.0, center=[29.717128543375022, -95.40287017822267], controls=(ZoomControl(options=['positio…

## Create bounding boxes

If no rectangles are drawn, the default bounding boxes will be used as follows:

In [ ]:
if m.user_rois is not None:
    boxes = m.user_rois
else:
    boxes = [
        [-51.2546, -22.1771, -51.2541, -22.1767],
        [-51.2538, -22.1764, -51.2535, -22.1761],
    ]

## Segment the image

Use the `predict()` method to segment the image with specified bounding boxes. The `boxes` parameter accepts a list of bounding box coordinates in the format of [[left, bottom, right, top], [left, bottom, right, top], ...], a GeoJSON dictionary, or a file path to a GeoJSON file.

In [ ]:
sam.predict(boxes=boxes, point_crs="EPSG:4326", output="mask.tif", dtype="uint8")

## Display the result

Add the segmented image to the map.

In [ ]:
m.add_raster("mask.tif", cmap="viridis", nodata=0, layer_name="Mask")
m

## Use an existing vector dataset as box prompts

You can also use an existing vector dataset as box prompts. The following example uses an existing dataset of tree bounding boxes from GitHub.

In [ ]:
geojson = (
    "https://github.com/opengeos/datasets/releases/download/samgeo/tree_boxes.geojson"
)

Display the bounding boxes on the map.

In [ ]:
m = leafmap.Map()
m.add_raster(image, layer_name="image")
style = {
    "color": "#ffff00",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0,
}
m.add_vector(
    geojson,
    style=style,
    zoom_to_layer=True,
    layer_name="Bounding boxes",
    info_mode=None,
)
m

## Segment trees with box prompts

Segment trees using the bounding boxes from the vector dataset.

In [ ]:
output_masks = "mask2.tif"
sam.predict(boxes=geojson, point_crs="EPSG:4326", output=output_masks, dtype="uint8")

Display the segmented masks on the map.

In [ ]:
m.add_raster(output_masks, nodata=0, opacity=0.5, layer_name="Tree masks")

## Post-processing

You can use the `region_groups()` method to clean up the segmentation results, such as removing small regions, and filling holes. In addition, you can compute geometric properties of the regions, such as area, perimeter, eccentricity, and solidity.

In [ ]:
out_image = "tree_masks.tif"
out_vector = "tree_vector.geojson"
array, gdf = sam.region_groups(
    output_masks, min_size=200, out_vector=out_vector, out_image=out_image
)

In [ ]:
gdf.head()

## Display the cleaned masks

In [ ]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
style = {
    "color": "#ffff00",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0,
}
m.add_raster(
    out_image, colormap="tab20", nodata=0, opacity=0.7, layer_name="Tree masks"
)
m.add_vector(out_vector, style=style, zoom_to_layer=True, layer_name="Tree vector")
m.add_vector(
    geojson,
    style={"color": "blue", "fillOpacity": 0},
    layer_name="Bounding boxes",
    info_mode=None,
)
m.add_layer_manager()
m

![image](https://github.com/user-attachments/assets/b789a0e6-6e76-4b10-a9b8-3fc14676481f)

## Create a split map

In [ ]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
m.split_map(
    out_image,
    image,
    left_label="Tree masks",
    right_label="Aerial imagery",
    left_args={"colormap": "tab20", "nodata": 0, "opacity": 0.7},
)
m

![demo](https://github.com/user-attachments/assets/7bb0a65c-94f1-4cb6-9361-e79b47ec1e0a)